In [18]:
"""
VALORANT STATISTICS SCRAPER - Fnatic Team Map Stats
This script extracts map statistics from VLR.gg website
"""

# Import necessary libraries
# requests: For getting the webpage
# BeautifulSoup: For parsing/reading the HTML content

import requests
from bs4 import BeautifulSoup
import csv  # Added for CSV export

# The URL of the webpage we want to scrape
# This is Fnatic's team stats page for a specific event
URL = [
    "https://www.vlr.gg/team/stats/8304/talon/?event_id=2277&series_id=all&core_id=all&date_start=&date_end=",
    "https://www.vlr.gg/team/stats/624/paper-rex/?event_id=2277&series_id=all&core_id=all&date_start=&date_end=",
    "https://www.vlr.gg/team/stats/278/detonation-focusme/?event_id=2277&series_id=all&core_id=all&date_start=&date_end=",
    "https://www.vlr.gg/team/stats/17/gen-g/?event_id=2277&series_id=all&core_id=all&date_start=&date_end=",
    "https://www.vlr.gg/team/stats/14/t1/?event_id=2277&series_id=all&core_id=all&date_start=&date_end=",
    "https://www.vlr.gg/team/stats/878/rex-regum-qeon/?event_id=2277&series_id=all&core_id=all&date_start=&date_end=",
    "https://www.vlr.gg/team/stats/8185/drx/?event_id=2277&series_id=all&core_id=all&date_start=&date_end=",
    "https://www.vlr.gg/team/stats/466/boom-esports/?event_id=2277&series_id=all&date_start=&date_end=",
    "https://www.vlr.gg/team/stats/918/global-esports/?event_id=2277&series_id=all&core_id=all&date_start=&date_end=",
    "https://www.vlr.gg/team/stats/5448/zeta-division/?event_id=2277&series_id=all&core_id=all&date_start=&date_end=",
]
# Send a request to the website to get the webpage content
# This is like opening the webpage in your browser
all_data = []
for link in URL:
    page = requests.get(link)
    
    # Parse the webpage content using BeautifulSoup
    # 'html.parser' tells BeautifulSoup to interpret the content as HTML
    # soup now contains the entire webpage structure that we can search through
    soup = BeautifulSoup(page.content, "html.parser")
    
    # FIND THE MAP STATISTICS TABLE
    # We're looking for a <table> element with specific class names
    # In HTML, 'class' is like a label that identifies what type of content it is
    # 'wf-table mod-team-maps' is the specific table that contains map statistics
    # Think of it like: "Find the table that has the label 'wf-table mod-team-maps'"
    team_name = link.split('/')[6]
    team_name = team_name.replace("-", " ").upper()
    maplist = soup.find_all('table', class_="wf-table mod-team-maps")
    
    # Extract just the key information for a clean summary
    if maplist:
        table = maplist[0]  # Get the first (and only) table
        
        # Find all data rows
        data_rows = table.find('tbody').find_all('tr')
        
        for row in data_rows:
            # Get map name from first cell
            data_row = {}
            first_cell = row.find('td')
            if first_cell:
                map_name_div = first_cell.find('div', style="justify-content: flex-start; white-space: nowrap;")
                if map_name_div:
                    map_name = map_name_div.get_text(strip=True)
                    name = map_name.split()[0]
                    pick = map_name.split()[1]

                    data_row['Team'] = team_name
                    data_row['Map'] = name
                    data_row['Pick_Type'] = pick
                    
                    # Get win percentage (should be in the 3rd data cell)
                    all_cells = row.find_all('td')
                    if len(all_cells) >= 3:
                        # The win % is inside a div with class 'mod-first'
                        win_cell = all_cells[2]
                        win_div = win_cell.find('div', class_='mod-first')
                        win_pct = win_div.get_text(strip=True) if win_div else "N/A"
                        
                        # Get wins and losses (cells 4 and 5)
                        wins = "N/A"
                        losses = "N/A"
                        
                        if len(all_cells) >= 5:
                            wins_cell = all_cells[3]
                            losses_cell = all_cells[4]
                            
                            wins_div = wins_cell.find('div', class_='mod-first')
                            losses_div = losses_cell.find('div', class_='mod-first')
                            
                            wins = wins_div.get_text(strip=True) if wins_div else "N/A"
                            losses = losses_div.get_text(strip=True) if losses_div else "N/A"
                            data_row['Wins'] = wins
                            data_row['Losses'] = losses
    
                            agent_names = []
                        if len(all_cells) > 13:  # Make sure we have enough cells
                            # Find all agent images in the agent composition cell
                            agent_images = all_cells[13].find_all('img')
                            
                            for img in agent_images:
                                # Extract agent name from image filename
                                img_src = img.get('src', '')
                                if img_src:
                                    # Extract filename from URL and remove .png extension
                                    agent_name = img_src.split('/')[-1].replace('.png', '')
                                    agent_names.append(agent_name)
                                    
                        data_row['Agents'] = ', '.join(agent_names)
                        all_data.append(data_row)
                        
                        # Print the formatted output
                        # print(f"\n• {name} pick = {pick}: {win_pct} win rate | {wins} wins, {losses} losses")
if all_data:
    # Define the CSV filename
    csv_filename = "valorant_team_stats.csv"
    
    # Define the column headers
    fieldnames = ['Team', 'Map', 'Pick_Type', 'Win_Percentage', 'Wins', 'Losses', 'Agents']
    
    # Open the CSV file for writing
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write the header row
        writer.writeheader()
        
        # Write all the data rows
        for row in all_data:
            writer.writerow(row)
    
    print("\n" + "=" * 50)
    print(f"✓ Data successfully saved to '{csv_filename}'")
    print(f"✓ Total records saved: {len(all_data)}")
    print("=" * 50)
else:
    print("\nNo data was scraped. CSV file was not created.")


✓ Data successfully saved to 'valorant_team_stats.csv'
✓ Total records saved: 120


In [16]:
"""
VALORANT Match Outcome Predictor using Machine Learning
Predicts which team would win between two given teams
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import warnings
warnings.filterwarnings('ignore')

# Load the data
df = pd.read_csv('valorant_team_stats.csv')

# Display basic info about the data
print("="*60)
print("VALORANT TEAM STATISTICS DATASET")
print("="*60)
print(f"Dataset shape: {df.shape}")
print(f"\nTeams in dataset: {df['Team'].nunique()}")
print(f"Maps in dataset: {df['Map'].nunique()}")
print("\nFirst few rows:")
print(df.head())
print("\n" + "="*60)

# Check data types and unique values
print("\nDATA EXPLORATION:")
print("="*60)
print("\nColumn data types:")
print(df.dtypes)
print(f"\nUnique values in Win_Percentage column:")
print(df['Win_Percentage'].unique()[:10])  # Show first 10 unique values
print(f"\nSample of Win_Percentage values:")
print(df['Win_Percentage'].head(10))

# DATA PREPROCESSING
print("\n" + "="*60)
print("PREPROCESSING DATA...")
print("="*60)

# Clean the data
df_clean = df.copy()

# First, let's handle the Win_Percentage column carefully
print("\nHandling Win_Percentage column...")

# Convert all values to string first, then clean
df_clean['Win_Percentage_str'] = df_clean['Win_Percentage'].astype(str)

# Remove % sign and convert to numeric
df_clean['Win_Percentage_clean'] = df_clean['Win_Percentage_str'].str.replace('%', '', regex=False)

# Convert to float, handling errors
df_clean['Win_Percentage_num'] = pd.to_numeric(df_clean['Win_Percentage_clean'], errors='coerce')

# Divide by 100 to get decimal
df_clean['Win_Percentage'] = df_clean['Win_Percentage_num'] / 100

# Drop the intermediate columns
df_clean = df_clean.drop(['Win_Percentage_str', 'Win_Percentage_clean', 'Win_Percentage_num'], axis=1)

# Fill NaN values with 0.5 (neutral win rate)
df_clean['Win_Percentage'] = df_clean['Win_Percentage'].fillna(0.5)

# Convert Wins and Losses to numeric
print("\nHandling Wins and Losses columns...")
df_clean['Wins'] = pd.to_numeric(df_clean['Wins'], errors='coerce').fillna(0).astype(int)
df_clean['Losses'] = pd.to_numeric(df_clean['Losses'], errors='coerce').fillna(0).astype(int)

print(f"\nAfter cleaning:")
print(f"Win_Percentage range: {df_clean['Win_Percentage'].min():.2f} to {df_clean['Win_Percentage'].max():.2f}")
print(f"Total rows: {len(df_clean)}")

# Calculate additional features
df_clean['Total_Games'] = df_clean['Wins'] + df_clean['Losses']
df_clean['Win_Rate'] = df_clean['Wins'] / df_clean['Total_Games'].replace(0, 1)  # Avoid division by zero

# Create feature for pick type (extract number from parentheses)
print("\nExtracting Pick_Count from Pick_Type...")
# Handle cases where Pick_Type might be NaN
df_clean['Pick_Type'] = df_clean['Pick_Type'].fillna('(0)')

# Extract number from parentheses using regex
df_clean['Pick_Count'] = df_clean['Pick_Type'].str.extract(r'\((\d+)\)').fillna('0').astype(int)

# Count number of agents in composition
print("\nCounting agents in composition...")
def count_agents(agent_str):
    if pd.isna(agent_str) or str(agent_str).strip() == '':
        return 0
    # Split by comma and count non-empty strings
    agents = [a.strip() for a in str(agent_str).split(',') if a.strip()]
    return len(agents)

df_clean['Agent_Count'] = df_clean['Agents'].apply(count_agents)

# Create a dictionary to store team statistics
print("\nCalculating team statistics...")
team_stats = {}

for team in df_clean['Team'].unique():
    team_data = df_clean[df_clean['Team'] == team]
    
    team_stats[team] = {
        'overall_win_rate': team_data['Win_Rate'].mean(),
        'total_wins': team_data['Wins'].sum(),
        'total_losses': team_data['Losses'].sum(),
        'total_games': team_data['Total_Games'].sum(),
        'avg_pick_count': team_data['Pick_Count'].mean(),
        'preferred_maps': team_data.groupby('Map')['Total_Games'].sum().sort_values(ascending=False).to_dict()
    }

print(f"Statistics calculated for {len(team_stats)} teams")

# CREATE TRAINING DATA FOR HEAD-TO-HEAD PREDICTIONS
print("\n" + "="*60)
print("CREATING TRAINING DATA FOR HEAD-TO-HEAD MATCHUPS...")
print("="*60)

# We'll create synthetic match data based on team statistics
# For each pair of teams and each map, we'll simulate potential outcomes

match_data = []
team_pairs = []

# Get all unique teams
all_teams = list(df_clean['Team'].unique())

print(f"Creating matchups for {len(all_teams)} teams...")

# Create synthetic matches between teams
for i, team1 in enumerate(all_teams):
    for j, team2 in enumerate(all_teams):
        if i < j:  # Avoid duplicate pairs and self-matches
            team_pairs.append((team1, team2))
            
            # Get each team's stats for each map
            team1_maps = df_clean[df_clean['Team'] == team1]
            team2_maps = df_clean[df_clean['Team'] == team2]
            
            # For each common map or all maps
            all_maps = set(team1_maps['Map']).union(set(team2_maps['Map']))
            
            for map_name in all_maps:
                # Get team1's stats on this map
                team1_map_stats = team1_maps[team1_maps['Map'] == map_name]
                team2_map_stats = team2_maps[team2_maps['Map'] == map_name]
                
                # If either team doesn't have stats for this map, use default values
                team1_win_rate = team1_map_stats['Win_Rate'].mean() if not team1_map_stats.empty else 0.5
                team1_games = team1_map_stats['Total_Games'].sum() if not team1_map_stats.empty else 0
                team1_pick = team1_map_stats['Pick_Count'].mean() if not team1_map_stats.empty else 0
                
                team2_win_rate = team2_map_stats['Win_Rate'].mean() if not team2_map_stats.empty else 0.5
                team2_games = team2_map_stats['Total_Games'].sum() if not team2_map_stats.empty else 0
                team2_pick = team2_map_stats['Pick_Count'].mean() if not team2_map_stats.empty else 0
                
                # Calculate probability of team1 winning (simplified)
                # Factors: win rate on map, number of games played, pick count
                team1_strength = (team1_win_rate * 0.6 + 
                                (min(team1_games, 10)/10) * 0.2 + 
                                (team1_pick/5) * 0.2)
                
                team2_strength = (team2_win_rate * 0.6 + 
                                (min(team2_games, 10)/10) * 0.2 + 
                                (team2_pick/5) * 0.2)
                
                # Determine outcome (team1 wins = 1, team2 wins = 0)
                # Add some randomness to simulate real matches
                if team1_strength > team2_strength + 0.1:  # Add threshold for clear wins
                    outcome = 1  # team1 wins
                elif team2_strength > team1_strength + 0.1:
                    outcome = 0  # team2 wins
                else:
                    # Close match, add some randomness
                    outcome = np.random.choice([0, 1], p=[0.5, 0.5])
                
                # Add features for machine learning
                match_features = {
                    'team1_overall_win_rate': team_stats[team1]['overall_win_rate'],
                    'team2_overall_win_rate': team_stats[team2]['overall_win_rate'],
                    'team1_map_win_rate': team1_win_rate,
                    'team2_map_win_rate': team2_win_rate,
                    'team1_map_games': team1_games,
                    'team2_map_games': team2_games,
                    'team1_pick_count': team1_pick,
                    'team2_pick_count': team2_pick,
                    'map_popularity': team1_games + team2_games,
                    'win_rate_diff': team1_win_rate - team2_win_rate,
                    'experience_diff': team1_games - team2_games,
                    'team1_total_wins': team_stats[team1]['total_wins'],
                    'team2_total_wins': team_stats[team2]['total_wins'],
                    'outcome': outcome
                }
                
                match_data.append(match_features)

# Convert to DataFrame
match_df = pd.DataFrame(match_data)

print(f"Created {len(match_df)} synthetic match records")
print(f"Team pairs: {len(team_pairs)}")
print("\nSample match data:")
print(match_df.head())

# PREPARE DATA FOR MACHINE LEARNING
print("\n" + "="*60)
print("PREPARING MACHINE LEARNING MODEL...")
print("="*60)

# Features and target
X = match_df.drop('outcome', axis=1)
y = match_df['outcome']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Check class distribution
print(f"\nClass distribution:")
print(y.value_counts())
print(f"Team1 wins: {(y == 1).sum()} ({((y == 1).sum()/len(y))*100:.1f}%)")
print(f"Team2 wins: {(y == 0).sum()} ({((y == 0).sum()/len(y))*100:.1f}%)")

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# TRAIN MULTIPLE MODELS
print("\n" + "="*60)
print("TRAINING MACHINE LEARNING MODELS...")
print("="*60)

models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(50, 25), max_iter=1000, random_state=42),
    'SVM': SVC(probability=True, random_state=42)
}

best_model = None
best_accuracy = 0
results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Evaluation
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Classification Report:")
    print(classification_report(y_test, y_pred, target_names=['Team2 Wins', 'Team1 Wins']))
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = name

print(f"\nBest model: {best_model} with accuracy: {best_accuracy:.4f}")

# SAVE THE BEST MODEL
print("\n" + "="*60)
print("SAVING THE BEST MODEL...")
print("="*60)

joblib.dump(results[best_model]['model'], 'valorant_predictor_model.pkl')
joblib.dump(scaler, 'valorant_predictor_scaler.pkl')
print("Model and scaler saved successfully!")

# PREDICTION FUNCTION
print("\n" + "="*60)
print("PREDICTION FUNCTION")
print("="*60)

def predict_match_outcome(team1_name, team2_name, map_name=None):
    """
    Predict the outcome of a match between two teams on a specific map
    """
    try:
        # Load model and scaler
        model = joblib.load('valorant_predictor_model.pkl')
        scaler = joblib.load('valorant_predictor_scaler.pkl')
    except:
        print("Error loading model. Using best model from current session.")
        model = results[best_model]['model']
    
    # Get team stats
    team1_data = df_clean[df_clean['Team'] == team1_name]
    team2_data = df_clean[df_clean['Team'] == team2_name]
    
    # If map is specified, get map-specific stats
    if map_name:
        team1_map_stats = team1_data[team1_data['Map'] == map_name]
        team2_map_stats = team2_data[team2_data['Map'] == map_name]
        
        team1_map_win_rate = team1_map_stats['Win_Rate'].mean() if not team1_map_stats.empty else 0.5
        team1_map_games = team1_map_stats['Total_Games'].sum() if not team1_map_stats.empty else 0
        team1_map_pick = team1_map_stats['Pick_Count'].mean() if not team1_map_stats.empty else 0
        
        team2_map_win_rate = team2_map_stats['Win_Rate'].mean() if not team2_map_stats.empty else 0.5
        team2_map_games = team2_map_stats['Total_Games'].sum() if not team2_map_stats.empty else 0
        team2_map_pick = team2_map_stats['Pick_Count'].mean() if not team2_map_stats.empty else 0
    else:
        # Use overall stats if no map specified
        team1_map_win_rate = team1_data['Win_Rate'].mean() if not team1_data.empty else 0.5
        team1_map_games = team1_data['Total_Games'].mean() if not team1_data.empty else 0
        team1_map_pick = team1_data['Pick_Count'].mean() if not team1_data.empty else 0
        
        team2_map_win_rate = team2_data['Win_Rate'].mean() if not team2_data.empty else 0.5
        team2_map_games = team2_data['Total_Games'].mean() if not team2_data.empty else 0
        team2_map_pick = team2_data['Pick_Count'].mean() if not team2_data.empty else 0
    
    # Get overall team stats
    team1_overall_win_rate = team_stats[team1_name]['overall_win_rate'] if team1_name in team_stats else 0.5
    team2_overall_win_rate = team_stats[team2_name]['overall_win_rate'] if team2_name in team_stats else 0.5
    team1_total_wins = team_stats[team1_name]['total_wins'] if team1_name in team_stats else 0
    team2_total_wins = team_stats[team2_name]['total_wins'] if team2_name in team_stats else 0
    
    # Prepare features for prediction
    features = pd.DataFrame([{
        'team1_overall_win_rate': team1_overall_win_rate,
        'team2_overall_win_rate': team2_overall_win_rate,
        'team1_map_win_rate': team1_map_win_rate,
        'team2_map_win_rate': team2_map_win_rate,
        'team1_map_games': team1_map_games,
        'team2_map_games': team2_map_games,
        'team1_pick_count': team1_map_pick,
        'team2_pick_count': team2_map_pick,
        'map_popularity': team1_map_games + team2_map_games,
        'win_rate_diff': team1_map_win_rate - team2_map_win_rate,
        'experience_diff': team1_map_games - team2_map_games,
        'team1_total_wins': team1_total_wins,
        'team2_total_wins': team2_total_wins,
    }])
    
    # Scale features and predict
    features_scaled = scaler.transform(features)
    prediction = model.predict(features_scaled)[0]
    probability = model.predict_proba(features_scaled)[0] if hasattr(model, 'predict_proba') else [0.5, 0.5]
    
    # Interpret result
    winner = team1_name if prediction == 1 else team2_name
    win_probability = probability[1] if prediction == 1 else probability[0]
    
    return {
        'team1': team1_name,
        'team2': team2_name,
        'map': map_name if map_name else 'Any Map',
        'predicted_winner': winner,
        'win_probability': win_probability,
        'team1_win_probability': probability[1],
        'team2_win_probability': probability[0],
        'team1_map_win_rate': team1_map_win_rate,
        'team2_map_win_rate': team2_map_win_rate,
        'team1_total_games': team1_map_games,
        'team2_total_games': team2_map_games
    }

def display_prediction_result(result):
    """Display prediction results in a readable format"""
    print("\n" + "="*60)
    print("MATCH PREDICTION RESULTS")
    print("="*60)
    print(f"Matchup: {result['team1']} vs {result['team2']}")
    print(f"Map: {result['map']}")
    print("\n" + "-"*60)
    print(f"PREDICTED WINNER: {result['predicted_winner']}")
    print(f"Win Probability: {result['win_probability']:.2%}")
    print("\n" + "-"*60)
    print("TEAM STATISTICS:")
    print(f"{result['team1']}:")
    print(f"  Map Win Rate: {result['team1_map_win_rate']:.2%}")
    print(f"  Games Played: {result['team1_total_games']}")
    print(f"  Win Probability: {result['team1_win_probability']:.2%}")
    print(f"\n{result['team2']}:")
    print(f"  Map Win Rate: {result['team2_map_win_rate']:.2%}")
    print(f"  Games Played: {result['team2_total_games']}")
    print(f"  Win Probability: {result['team2_win_probability']:.2%}")
    
    # Show recommendation
    print("\n" + "-"*60)
    if result['team1_win_probability'] > 0.65:
        print(f"STRONG FAVORITE: {result['team1']}")
    elif result['team2_win_probability'] > 0.65:
        print(f"STRONG FAVORITE: {result['team2']}")
    elif abs(result['team1_win_probability'] - result['team2_win_probability']) < 0.1:
        print("CLOSE MATCH - Could go either way!")
    else:
        print(f"SLIGHT FAVORITE: {result['predicted_winner']}")
    print("="*60)

# INTERACTIVE PREDICTION
print("\n" + "="*60)
print("INTERACTIVE PREDICTION MODE")
print("="*60)

# Example predictions
example_matches = [
    ('TALON', 'PAPER REX', 'Bind'),
    ('GEN G', 'DRX', 'Lotus'),
    ('T1', 'GEN G', None),  # No specific map
    ('DRX', 'PAPER REX', 'Fracture'),
]

for team1, team2, map_name in example_matches:
    result = predict_match_outcome(team1, team2, map_name)
    display_prediction_result(result)

# Function to get user input for predictions
def interactive_predictor():
    print("\n" + "="*60)
    print("CUSTOM MATCH PREDICTION")
    print("="*60)
    
    available_teams = sorted(df_clean['Team'].unique())
    available_maps = sorted(df_clean['Map'].unique())
    
    print(f"Available teams ({len(available_teams)}):")
    for i, team in enumerate(available_teams, 1):
        print(f"  {team}", end=' | ' if i % 3 != 0 else '\n')
    print()
    
    print(f"Available maps ({len(available_maps)}):")
    for i, map_name in enumerate(available_maps, 1):
        print(f"  {map_name}", end=' | ' if i % 3 != 0 else '\n')
    print()
    
    while True:
        print("\nEnter match details (or 'quit' to exit):")
        
        team1 = input("Enter first team name: ").strip().upper()
        if team1.lower() == 'quit':
            break
        
        team2 = input("Enter second team name: ").strip().upper()
        if team2.lower() == 'quit':
            break
        
        map_choice = input("Enter map name (or press Enter for any map): ").strip()
        map_name = map_choice if map_choice else None
        
        if team1 in available_teams and team2 in available_teams:
            if map_name and map_name not in available_maps:
                print(f"Warning: Map '{map_name}' not found in dataset. Using overall stats.")
                map_name = None
            
            if team1 == team2:
                print("Error: Cannot predict match between the same team!")
                continue
                
            result = predict_match_outcome(team1, team2, map_name)
            display_prediction_result(result)
        else:
            print("Error: One or both teams not found in dataset.")
            if team1 not in available_teams:
                print(f"  '{team1}' not found.")
            if team2 not in available_teams:
                print(f"  '{team2}' not found.")

# Run interactive predictor
interactive_predictor()

print("\n" + "="*60)
print("VALORANT MATCH PREDICTOR COMPLETE!")
print("="*60)
print("\nTo make predictions later, run:")
print("""
# Load and use the predictor
result = predict_match_outcome('TEAM1', 'TEAM2', 'MapName')
display_prediction_result(result)
""")
print("Example:")
print("  predict_match_outcome('TALON', 'PAPER REX', 'Bind')")
print("  predict_match_outcome('GEN G', 'DRX', None)  # Any map")
print("\n" + "="*60)

VALORANT TEAM STATISTICS DATASET
Dataset shape: (120, 7)

Teams in dataset: 10
Maps in dataset: 12

First few rows:
    Team     Map Pick_Type  Win_Percentage  Wins  Losses  \
0  TALON    Bind       (3)             NaN     2       1   
1  TALON   Haven       (1)             NaN     0       1   
2  TALON   Split       (2)             NaN     1       1   
3  TALON  Ascent       (0)             NaN     0       0   
4  TALON  Icebox       (0)             NaN     0       0   

                                              Agents  
0                 astra, deadlock, fade, gekko, raze  
1                   breach, cypher, jett, omen, sova  
2  astra, cypher, deadlock, raze, skye, astra, br...  
3                                                NaN  
4                                                NaN  


DATA EXPLORATION:

Column data types:
Team               object
Map                object
Pick_Type          object
Win_Percentage    float64
Wins                int64
Losses              int

Enter first team name:  DRX
Enter second team name:  T1
Enter map name (or press Enter for any map):  



MATCH PREDICTION RESULTS
Matchup: DRX vs T1
Map: Any Map

------------------------------------------------------------
PREDICTED WINNER: DRX
Win Probability: 59.14%

------------------------------------------------------------
TEAM STATISTICS:
DRX:
  Map Win Rate: 40.28%
  Games Played: 1.0833333333333333
  Win Probability: 59.14%

T1:
  Map Win Rate: 30.56%
  Games Played: 1.8333333333333333
  Win Probability: 40.86%

------------------------------------------------------------
SLIGHT FAVORITE: DRX

Enter match details (or 'quit' to exit):


Enter first team name:  quit



VALORANT MATCH PREDICTOR COMPLETE!

To make predictions later, run:

# Load and use the predictor
result = predict_match_outcome('TEAM1', 'TEAM2', 'MapName')
display_prediction_result(result)

Example:
  predict_match_outcome('TALON', 'PAPER REX', 'Bind')
  predict_match_outcome('GEN G', 'DRX', None)  # Any map

